In [53]:
class TrieNode:
    def __init__(self, ch):
        self.ch = ch
        self.childrens = {}
        self.isEndOfWord = False
    

class Trie:

    def __init__(self):
        self.root = TrieNode("*")
        

    def insert(self, word: str) -> None:
        current = self.root
        for ch in word:
            if ch not in current.childrens:
                current.childrens[ch] = TrieNode(ch)
            current = current.childrens[ch]
        current.isEndOfWord = True
        
    def insertPara(self, para):
        for wrd in para.split(" "):
            if wrd:
                self.insert(wrd.strip())

    def search(self, word: str) -> bool:
        current = self.root
        for ch in word:
            if ch not in current.childrens:
                return False
            current = current.childrens[ch]
        return current.isEndOfWord

    def startsWith(self, prefix: str) -> bool:
        current = self.root
        for ch in prefix:
            if ch not in current.childrens:
                return False
            current = current.childrens[ch]
        return True
    
    def print(self):
        printNode("", self.root, True)
        
def printNode(prefix, node, isLast):
    if node is not None:
        print(prefix + "+- " + str(node.ch) + ("." if node.isEndOfWord else "") )
        length = len(node.childrens)
        count = 0
        for ch in node.childrens:
            count += 1
            printNode(prefix + (" " if isLast else "| ") , node.childrens[ch] , count == length)
            
            
trie = Trie()
para = """
apple , ball , aeroplane , air , arid
"""
trie.insertPara(para.lower())
trie.print()
trie.startsWith("ai")

+- *
 +- a
 | +- p
 | | +- p
 | |  +- l
 | |   +- e.
 | +- e
 | | +- r
 | |  +- o
 | |   +- p
 | |    +- l
 | |     +- a
 | |      +- n
 | |       +- e.
 | +- i
 | | +- r.
 | +- r
 |  +- i
 |   +- d.
 +- ,.
 +- b
  +- a
   +- l
    +- l.


True

In [36]:
class PatternSearchTrie(Trie):
    
    def _internalSeach(self, word, start , current):
        for i in range(start, len(word)):
            ch = word[i]
            if ch == ".":
                for child in current.childrens:
                    if self._internalSeach(word, i + 1, current.childrens[child]):
                        return True
                return False
            if ch not in current.childrens:
                return False
            current = current.childrens[ch]
        return current.isEndOfWord
    
    def searchPattern(self, word: str) -> bool:
        return self._internalSeach(word, 0, self.root)
    
trie = PatternSearchTrie()
para = "ball cat bad dad "
trie.insertPara(para.lower())
trie.print()

trie.searchPattern(".cd")

+- *
 +- b
 | +- a
 |  +- l
 |  | +- l
 |  +- d
 +- c
 | +- a
 |  +- t
 +- d
  +- a
   +- d


False

In [89]:
def splitCombination(words, taken, seq, splits = []):
    
    if taken == len(words):
        splits.append(seq.split(","))
        return splits
    
    for i in range(taken + 1, len(words) + 1):
        ss = words[taken:i]
        splitCombination(words, i, seq + "," + ss, splits)
        
    return splits
    
import numpy as np


def wordBreak(words, wordDict) -> bool:
    trie = Trie()
    for word in wordDict: trie.insert(word)
#     trie.print()
    
    splits = splitCombination(words, 0 , "")
#     for i in splits:
#         print(i)
    for seq in splits:
        seqBool = True
        for i in range(1, len(seq)):
            if not trie.search(seq[i]):
                seqBool = False
                break
        if seqBool:
            print(seq)
            return seqBool
    return False




s = "acccbccb"
wordDict = ["cc","bc","ac","ca"]
wordBreak(s, wordDict)
# splitCombination(s, 0 , "")

False

In [146]:
from collections import defaultdict

def findWords(board, words):
    firstChars = defaultdict(lambda : [])
    for i in range(len(words)):
        firstChars[words[i][0]].append(i)
    print(firstChars)
    items = defaultdict(lambda : [])
    for i in range(len(board)):
        for j in range(len(board[i])):
            if board[i][j] in firstChars:
                wordIndexs = firstChars[board[i][j]]
                for wordIndex in wordIndexs:
                    items[wordIndex].append((i, j))
    print(items)
    output = set()
    
    for k in items:
        wrd = words[k]
        places_to_look = items[k]
        print(wrd, places_to_look)
        for i,j in places_to_look:
            visited = set()
            search_rec(board, wrd, 0, i, j, visited, output)
            
    print(output)
    return output

def search_itr(board, wrd, wordIndex, i, j, visited, output):
    stack = [(i, j, 0)]
    while len(stack) > 0:
        i, j , wordIndex = stack[-1]
        visited.add((i,j))
        print("stack", stack, board[i][j], wrd[wordIndex] )
        if wordIndex + 1 >= len(wrd):
            output.add(wrd)
            break
        l = len(stack)
        for mi,mj in movements:
            xi = i + mi
            xj = j + mj
            if 0 <= xi < len(board) and 0 <= xj < len(board[i]) and (xi, xj) not in visited:
                if wrd[wordIndex + 1] == board[xi][xj]:
                    stack.append((xi, xj, wordIndex + 1))
        if len(stack) == l:
            while len(stack) > 0 and (stack[-1][0],stack[-1][1]) in visited:
                visited.remove((stack[-1][0],stack[-1][1]))
                stack.pop()

movements = [(1,0),(-1,0),(0,1),(0,-1)]

def search_rec(board, wrd, wordIndex, i, j, visited, output):
    visited.add((i,j))
    if wordIndex + 1 >= len(wrd):
        output.add(wrd)
        return
    for mi,mj in movements:
        xi = i + mi
        xj = j + mj
        if 0 <= xi < len(board) and 0 <= xj < len(board[i]) and (xi, xj) not in visited:
            if wrd[wordIndex + 1] == board[xi][xj]:
                search_rec(board, wrd, wordIndex + 1 , xi , xj , visited, output)

    visited.remove((i,j))


board = [
    ["o","a","a","n"],
    ["e","t","a","e"],
    ["i","h","k","r"],
    ["i","f","l","v"]
]
words = ["oath","pea","eat","rain"]
findWords(board, words)

board = [["a","b","e"],
         ["b","c","d"]]
words = ["abcdeb"]



findWords(board, words)

defaultdict(<function findWords.<locals>.<lambda> at 0x00000273D6BE6B80>, {'o': [0], 'p': [1], 'e': [2], 'r': [3]})
defaultdict(<function findWords.<locals>.<lambda> at 0x00000273D6D4FE50>, {0: [(0, 0)], 2: [(1, 0), (1, 3)], 3: [(2, 3)]})
oath [(0, 0)]
eat [(1, 0), (1, 3)]
rain [(2, 3)]
{'eat', 'oath'}
defaultdict(<function findWords.<locals>.<lambda> at 0x00000273D6BE6B80>, {'a': [0]})
defaultdict(<function findWords.<locals>.<lambda> at 0x00000273D6D4F550>, {0: [(0, 0)]})
abcdeb [(0, 0)]
{'abcdeb'}


{'abcdeb'}

In [153]:
def findWords(board, words):
    trie = Trie()
    for i in range(len(words)):
        trie.insert(words[i])
    trie.print()
    output = set()
    for i in range(len(board)):
        for j in range(len(board[i])):
            if board[i][j] in trie.root.childrens:
                checkTrie(board, i, j, board[i][j] ,trie.root.childrens[board[i][j]], output)
            
    return output

movements = [(1,0),(-1,0),(0,1),(0,-1)]

def checkTrie(board, i, j, prefix, currentNode, output):
    if currentNode.isEndOfWord:
        output.add(prefix)
    tmp = board[i][j]
    board[i][j] = "$"
    for mi,mj in movements:
        xi = i + mi
        xj = j + mj
        if 0 <= xi < len(board) and 0 <= xj < len(board[i]):
            if board[xi][xj] in currentNode.childrens:
                checkTrie(board, xi ,xj , prefix + board[xi][xj], currentNode.childrens[board[xi][xj]], output)
    board[i][j] = tmp

board = [
    ["o","a","a","n"],
    ["e","t","a","e"],
    ["i","h","k","r"],
    ["i","f","l","v"]
]
words = ["oath","pea","eat","rain"]

board = [["o","a","b","n"],["o","t","a","e"],["a","h","k","r"],["a","f","l","v"]]
words = ["oa","oaa"]


findWords(board, words)

+- *
 +- o
  +- a.
   +- a.


{'oa', 'oaa'}